In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/deeplab-demo/plan-dease.zip'

extract_path = '/content/data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

extracted_files = os.listdir(extract_path)
import pandas as pd

train_csv_path = '/content/data/train.csv'

train_df = pd.read_csv(train_csv_path)


In [ ]:
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import os
sample_data = train_df.sample(32)
images = [load_and_preprocess_image(img_id) for img_id in sample_data['image_id']]
images = np.array(images)

label_column = 'healthy'
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(sample_data[label_column].values)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)),  # Điều chỉnh input_shape
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

model.fit(images, labels, epochs=3)

In [ ]:
import torch
from torchvision.models import densenet121
from torch import nn

class DeepLabV3DenseNet121(nn.Module):
    def __init__(self, num_classes=4):
        super(DeepLabV3DenseNet121, self).__init__()
        densenet = densenet121(pretrained=True).features
        self.backbone = nn.Sequential(*list(densenet.children())[:-1])
        self.classifier = nn.Sequential(
            nn.Conv2d(1024, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, num_classes, kernel_size=1)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

model = DeepLabV3DenseNet121(num_classes=4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model_path = '/content/drive/MyDrive/deeplab-demo/model/deeplabv3_densenet121.pth'
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

In [ ]:
import torch
from torchvision.models import densenet121
from torch import nn

class DeepLabV3DenseNet121(nn.Module):
    def __init__(self, num_classes=4):
        super(DeepLabV3DenseNet121, self).__init__()
        densenet = densenet121(pretrained=True).features # encoder
        self.backbone = nn.Sequential(*list(densenet.children())[:-1])
        # decoder
        self.classifier = nn.Sequential(
            nn.Conv2d(1024, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, num_classes, kernel_size=1)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

model = DeepLabV3DenseNet121()

print(model)

# backbone
for name, layer in model.backbone.named_modules():
    print(f"Layer name: {name}")
    print(layer)
    if isinstance(layer, nn.Conv2d):
        print(f" - Number of filters: {layer.out_channels}")
        print(f" - Kernel size: {layer.kernel_size}")
        print(f" - Stride: {layer.stride}")
        print(f" - Padding: {layer.padding}")
    print()

# hông tin chi tiết layer trong classifier
for name, layer in model.classifier.named_modules():
    print(f"Layer name: {name}")
    print(layer)
    if isinstance(layer, nn.Conv2d):
        print(f" - Number of filters: {layer.out_channels}")
        print(f" - Kernel size: {layer.kernel_size}")
        print(f" - Stride: {layer.stride}")
        print(f" - Padding: {layer.padding}")
    print()